In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)


In [4]:
df = pd.read_csv("data/df_merged.csv", delimiter=",", index_col= False)
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'tweet', 'entity list', 'target',
       'severity', 'reason', 'muhattap', 'mobil', 'sabit', 'ek paket',
       'kampanya', 'diğer\nürün / uygulama / cihaz', 'reklam', 'çağrı merkezi',
       'bayi', 'aksiyon'],
      dtype='object')

In [5]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0','target',
       'severity', 'reason', 'muhattap', 'mobil', 'sabit', 'ek paket',
       'kampanya', 'diğer\nürün / uygulama / cihaz', 'reklam', 'çağrı merkezi',
       'bayi', 'aksiyon'], axis=1)

In [7]:
df = df.rename(columns={"tweet": "tweet", "entity list": "entities"})

In [8]:
entity_list = df.entities.to_list()
tweet_list = df.tweet.to_list()

In [9]:
def tagNormalize(tags):
    return [i.lstrip().rstrip().lower()for i in tags]


In [79]:
def findSubstringInText(text, words, tags):
    text = text.lower()
    temp = []
    startpos = 0
    for idx, t in enumerate(words):
        spos =text.find(t, startpos) 
        if spos >= 0:
            epos = spos+len(t)
            #startpos = epos
            temp.append([spos, epos, tags[idx]])
    return temp

In [11]:
for idx, entitiy in enumerate(entity_list[0:5]):
    
    lowertext = tweet_list[idx].lower()
    ent= tagNormalize(entitiy.split(";"))
    rs = findSubstringInText(tweet_list[idx], ent)
    print(rs)
    

[[41, 49, 'turkcell']]
[[40, 48, 'turkcell']]
[[16, 27, 'superonline']]
[[0, 8, 'turkcell']]
[[0, 8, 'turkcell'], [9, 12, 'kaş']]


In [12]:
full_ent = []
for idx, entitiy in enumerate(entity_list):
    normal_ent= tagNormalize(entitiy.split(";"))
    for e in normal_ent:
        full_ent.append(e.lstrip().rstrip().lower())

In [86]:
with open ("entities.csv", "w+") as file:
    for e in set(full_ent):
        file.writelines(e+"\n")

In [24]:
df = pd.read_csv("entities_filtered.csv", index_col= False)
df

,word,ent
0,mobil hat,PRODUCT
1,turkcellhizmet,APP
2,müşteri hizmetleri,HIZMET
3,faturasız hat,PRODUCT
4,fibernet,PRODUCT
...,...,...
534,apple store,APP
535,twitter paketi,PACKAGE
536,finansman kredisi,HIZMET
537,teledünya,PACKAGE


In [25]:
df = df.drop_duplicates(subset=['word'])
df

,word,ent
0,mobil hat,PRODUCT
1,turkcellhizmet,APP
2,müşteri hizmetleri,HIZMET
3,faturasız hat,PRODUCT
4,fibernet,PRODUCT
...,...,...
534,apple store,APP
535,twitter paketi,PACKAGE
536,finansman kredisi,HIZMET
537,teledünya,PACKAGE


In [29]:
df.to_csv("entities_filtered_last.csv", index= False)

In [116]:
df = pd.read_csv("entities_filtered_last.csv", index_col= False)
df

,word,ent
0,3g,HIZMET
1,4.5 g,HIZMET
2,4.5g,HIZMET
3,4g,HIZMET
4,5g,HIZMET
...,...,...
417,yurt dışı,HIZMET
418,yurt dışı kullanım,HIZMET
419,yurt dışı paketi,PACKAGE
420,yurt dışında kullanım,PACKAGE


In [117]:
new_word_list = df.word.to_list()
new_tag_list= df.ent.to_list()

tag_full = []
tag_operator = df[df['ent']=="OPERATOR"].word.to_list()
tag_app = df[df['ent']=="APP"].word.to_list()
tag_hizmet = df[df['ent']=="HIZMET"].word.to_list()
tag_product = df[df['ent']=="PRODUCT"].word.to_list()
tag_package = df[df['ent']=="PACKAGE"].word.to_list()


In [118]:
df.ent.unique()

array(['HIZMET', 'APP', 'OPERATOR', 'PRODUCT', 'PACKAGE'], dtype=object)

In [119]:
tag_full.append(tag_app)
tag_full.append(tag_hizmet)
tag_full.append(tag_operator)
tag_full.append(tag_package)
tag_full.append(tag_operator)

In [120]:
tag_str = ['APP', 'HIZMET', 'OPERATOR', 'PACKAGE', 'PRODUCT']

In [133]:
def findSubstringInText(text):
    text = text.lower()
    temp = []
    startpos = 0
    for index, tf in enumerate(tag_full):
        for idx, t in enumerate(tf):
            spos =text.find(t, startpos) 
            if spos >= 0:
                epos = spos+len(t)
                startpos = epos
                temp.append([spos, epos, tag_str[index]])
        
    return temp

In [152]:
import json
ner_data = []
for idx, tweet in enumerate(tweet_list):
    labels = findSubstringInText(tweet.lower())
    ner_data.append({"id": idx, "text": tweet, "label": labels})

with open ("ner_data.json", "w+") as nerfile:
    for n in ner_data:
        nerfile.writelines(json.dumps(n, ensure_ascii= False)+"\n")

In [31]:
for idx, entitiy in enumerate(tweet_list[0:5]):
    
    lowertext = tweet_list[idx].lower()
    ent= tagNormalize(entitiy.split(";"))
    rs = findSubstringInText(tweet_list[idx], ent)
    
    print(entitiy)

Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkcell internet paketin bitti diye nasıl mesaj atabilir bana ya? Onu da mı ödeyelim
20 dk 1 GB internet 500 mb sadece kaşar turkcell de düşer. Oç çocukları.
Ayrıca turkcell superonline reklamı kadar da kötü bir reklam görmemiştim
Turkcell çok pahalı ya
Turkcell Kaş'ta internetin cekmiyor
